<a href="https://colab.research.google.com/github/nitish-raj/Price-Predictions-ML/blob/master/Algorithmic_Trading_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Description**
This program uses following two methonds to determine when to buy and sell stock:


1. The dual simple moving average crossover (SMA 30 vs SMA 100)
2. Moving Average Convergence/Divergence (MACD) Cross Over
3. Three moving average
4. Money Flow Index(MFI)



In [ ]:
# Import the liraries
import pandas as pd
import numpy as np
import pandas_datareader as web
from datetime import datetime
import matplotlib.pyplot as plt

plt.style.use("fivethirtyeight")

In [ ]:
# Load the data
# Get the stock quote
stock = web.DataReader("AMZN", data_source="yahoo", start="2014-04-01", end="2021-04-30")
# Show data
stock

In [ ]:
# Visualize the data
plt.figure(figsize=(16, 6))
plt.plot(AAPL["Adj Close"], label="AAPL")
plt.title("Apple Adj. Close Price History")
plt.xlabel("Oct. 02. 2014 - May 40 2021")
plt.ylabel("Adj. Close price USD ($)")
plt.legend(loc="upper left")
plt.show()

## **Strategy 1 : SMA**

In [ ]:
# Create the simple moving average with 30 day window
SMA30 = pd.DataFrame()
SMA30["Adj Close Price"] = AAPL["Adj Close"].rolling(window=30).mean()
SMA30

In [ ]:
# Create a simple moving 100 day average
SMA100 = pd.DataFrame()
SMA100["Adj Close Price"] = AAPL["Adj Close"].rolling(window=100).mean()
SMA100

In [ ]:
# Visualize the data
plt.figure(figsize=(20, 8))
plt.plot(AAPL["Adj Close"], label="AAPL")
plt.plot(SMA30["Adj Close Price"], label="SMA30")
plt.plot(SMA100["Adj Close Price"], label="SMA100")
plt.title("Apple Adj. Close Price History")
plt.xlabel("Oct. 02. 2006 - May 40 2021")
plt.ylabel("Adj. Close price USD ($)")
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Create a new data frame to store all the data
data = pd.DataFrame()
data["AAPL"] = AAPL["Adj Close"]
data["SMA30"] = SMA30["Adj Close Price"]
data["SMA100"] = SMA100["Adj Close Price"]
data

In [ ]:
# Create function to signal when to buy and sell the asset/ stock
def buy_sell(data):
    sigPriceBuy = []
    sigPriceSell = []
    flag = -1

    for i in range(len(data)):
        if data["SMA30"][i] > data["SMA100"][i]:
            if flag != 1:
                sigPriceBuy.append(data["AAPL"][i])
                sigPriceSell.append(np.nan)
                flag = 1
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        elif data["SMA30"][i] < data["SMA100"][i]:
            if flag != 0:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(data["AAPL"][i])
                flag = 0
            else:
                sigPriceBuy.append(np.nan)
                sigPriceSell.append(np.nan)
        else:
            sigPriceBuy.append(np.nan)
            sigPriceSell.append(np.nan)
    return (sigPriceBuy, sigPriceSell)

In [ ]:
# Store the buy and sell data into
buy_sell = buy_sell(data)
data["Signal_buy_Price"] = buy_sell[0]
data["Signal_sell_Price"] = buy_sell[1]

In [ ]:
# Show the data
data

In [ ]:
# Visualize the data and strategy to buy and sell stock
plt.figure(figsize=(20, 8))
plt.plot(data["AAPL"], label="AAPL", alpha=0.35)
plt.plot(data["SMA30"], label="SMA30", alpha=0.35)
plt.plot(data["SMA100"], label="SMA100", alpha=0.35)
plt.scatter(
    data.index, data["Signal_buy_Price"], label="Buy", marker="^", color="green"
)
plt.scatter(
    data.index, data["Signal_sell_Price"], label="Sell", marker="v", color="red"
)
plt.title("Apple Adj. Close Price History & Buy and Sell Signals")
plt.xlabel("Apr. 01. 2014 - May 40 2021")
plt.ylabel("Adj. Close price USD ($)")
plt.legend(loc="upper left")
plt.show()

## **Strategy 2: MACD**


In [ ]:
# Load the data
# Get the stock quote
AAPL = web.DataReader(
    "PNB.NS", data_source="yahoo", start="2021-02-01", end="2021-05-18"
)
# Show data
AAPL

In [ ]:
plt.figure(figsize=(12.5, 4.5))
plt.plot(AAPL["Close"], label="Close")
plt.xticks(rotation=45)
plt.title("Close Price History")
plt.xlabel("Date")
plt.ylabel("Price USD ($)")
plt.show()

In [ ]:
# Calculate the MACD and Signal line indicators
# Calculated the short term exponential moving average (EMA)
ShortEMA = AAPL.Close.ewm(span=12, adjust=False).mean()
# Calculate the long term exponential moving average
LongEMA = AAPL.Close.ewm(span=26, adjust=False).mean()
# Calculate the MACD line
MACD = ShortEMA - LongEMA
# Calculate the signal line
signal = MACD.ewm(span=9, adjust=False).mean()

In [ ]:
# Plot the chart
plt.figure(figsize=(20, 8))
plt.plot(AAPL.index, MACD, label="AAPL MACD", color="red")
plt.plot(AAPL.index, signal, label="Signal Line", color="blue")
plt.xticks(rotation=45)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Creaet new columns for the data
AAPL["MACD"] = MACD
AAPL["Signal Line"] = signal

# Show data
AAPL

In [ ]:
# Create a function to signal whn to buy and sell and asset
def buy_sell(signal):
    Buy = []
    Sell = []
    flag = -1

    for i in range(len(signal)):
        if signal["MACD"][i] > signal["Signal Line"][i]:
            Sell.append(np.nan)
            if flag != 1:
                Buy.append(signal["Close"][i])
                flag = 1
            else:
                Buy.append(np.nan)
        elif signal["MACD"][i] < signal["Signal Line"][i]:
            Buy.append(np.nan)
            if flag != 0:
                Sell.append(signal["Close"][i])
                flag = 0
            else:
                Sell.append(np.nan)
        else:
            Sell.append(np.nan)
            Buy.append(np.nan)
    return Buy, Sell

In [ ]:
# Create buy and sell column
a = buy_sell(AAPL)
AAPL["Buy signal Price"] = a[0]
AAPL["Sell Signal Price"] = a[1]
AAPL

In [ ]:
AAPL

In [ ]:
# Visualize the data and strategy to buy and sell stock
plt.figure(figsize=(20, 8))
plt.plot(AAPL["Close"], label="Close Price", alpha=0.35)
plt.scatter(
    AAPL.index,
    AAPL["Buy signal Price"],
    label="Buy",
    marker="^",
    color="green",
    alpha=1,
)
plt.scatter(
    AAPL.index, AAPL["Sell Signal Price"], label="Sell", marker="v", color="red"
)
plt.title("Apple Close Price History & Buy and Sell Signals")
plt.xticks(rotation=45)
plt.xlabel("Date")
plt.ylabel("Close price USD ($)")
plt.legend(loc="upper left")
plt.show()

## **Strategy 3: Three Moving Average**

In [ ]:
# Load the data
# Get the stock quote
df = web.DataReader("TSLA", data_source="yahoo", start="2020-01-01", end="2021-05-30")
# Show data
df

In [ ]:
# Visualize the data
plt.figure(figsize=(20, 4))
plt.title("Close Price")
plt.plot(df["Close"], label="TSLA CLose Price")
plt.xlabel("Date", fontsize=14)
plt.ylabel("Close Price ($)", fontsize=14)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Calculate the three moving averages

# Calculate the shoft/fast exponenetial moving average
df["ShortEMA"] = df.Close.ewm(span=5, adjust=False).mean()

# Calculate the Middle/Medium exponential moving averages
df["MiddleEMA"] = df.Close.ewm(span=21, adjust=False).mean()

# Calculate the long/slow exponential moving average
df["LongEMA"] = df.Close.ewm(span=63, adjust=False).mean()

In [ ]:
# Visualiuze the closing price and the exponential moving averages
# Visualize the data
plt.figure(figsize=(25, 10))
plt.title("TSLA : Close Price vs Short EMA vs MiddleEMA vs LongEMA")
plt.plot(df["Close"], label="TSLA CLose Price", color="blue")
plt.plot(df["ShortEMA"], label="ShortEMA", color="red")
plt.plot(df["MiddleEMA"], label="MiddleEMA", color="orange")
plt.plot(df["LongEMA"], label="LongEMA", color="green")
plt.xlabel("Date", fontsize=14)
plt.xticks(rotation=45)
plt.ylabel("Close Price ($)", fontsize=14)
plt.legend(loc="upper left", fontsize=16)
plt.show()

In [ ]:
# Create the function to buy and sell the stock


def buy_sell_function(data):
    buy_list = []
    sell_list = []
    flag_long = False
    flag_short = False

    for i in range(0, len(data)):
        if (
            (data["MiddleEMA"][i] < data["LongEMA"][i])
            and (data["ShortEMA"][i] < data["MiddleEMA"][i])
            and flag_long == False
            and flag_short == False
        ):
            buy_list.append(data["Close"][i])
            sell_list.append(np.nan)
            flag_short = True
        elif flag_short == True and data["ShortEMA"][i] > data["MiddleEMA"][i]:
            sell_list.append(data["Close"][i])
            buy_list.append(np.nan)
            flag_short = False
        elif (
            data["MiddleEMA"][i] > data["LongEMA"][i]
            and data["ShortEMA"][i] > data["MiddleEMA"][i]
            and flag_long == False
            and flag_short == False
        ):
            buy_list.append(data["Close"][i])
            sell_list.append(np.nan)
            flag_long = True
        elif flag_long == True and data["ShortEMA"][i] < data["MiddleEMA"][i]:
            sell_list.append(data["Close"][i])
            buy_list.append(np.nan)
            flag_long = False
        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)
    return (buy_list, sell_list)

In [ ]:
df["Buy"] = buy_sell_function(df)[0]
df["Sell"] = buy_sell_function(df)[1]

In [ ]:
df

In [ ]:
# Visualiuze the closing price and the exponential moving averages
# Visualize the data
plt.figure(figsize=(25, 10))
plt.title("TSLA : Buy and Sell Chart")
plt.plot(df["Close"], label="TSLA CLose Price", color="blue", alpha=0.35)
# plt.plot(df['ShortEMA'],label = 'Short/Fast EMA',color = 'red',alpha = 0.35)
# plt.plot(df['MiddleEMA'],label = 'Middle/Medium EMA', color = 'orange',alpha = 0.35)
# plt.plot(df['LongEMA'],label = 'Long/Slow EMA',color = 'green',alpha = 0.35)
plt.scatter(df.index, df["Buy"], color="green", marker="^", alpha=1, s=100)
plt.scatter(df.index, df["Sell"], color="red", marker="v", alpha=1, s=100)
plt.xlabel("Date", fontsize=14)
plt.xticks(rotation=45)
plt.ylabel("Close Price ($)", fontsize=14)
plt.legend(loc="upper left", fontsize=16)
plt.show()

## **Strategy 4: MFI**

In [ ]:
# Load the data
# Get the stock quote
df = web.DataReader("AAPL", data_source="yahoo", start="2020-01-01", end="2021-05-30")
# Show data
df